Step 1:- Import Required Libraries

In [ ]:
!pip uninstall -y huggingface_hub

Found existing installation: huggingface_hub 1.4.0
Uninstalling huggingface_hub-1.4.0:
  Successfully uninstalled huggingface_hub-1.4.0


In [ ]:
!pip install -U huggingface_hub>=0.35.0

In [ ]:
Step 2:-Hugging face Authentication

In [1]:
from huggingface_hub import login
login()

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Step 3 :-Verify who

In [ ]:
from huggingface_hub import whoami
whoami()

{'type': 'user',
 'id': '69745659195d4e2c6d38819f',
 'name': 'lakshmibalaji',
 'fullname': 'M',
 'canPay': False,
 'billingMode': 'prepaid',
 'periodEnd': 1772323200,
 'isPro': False,
 'avatarUrl': '/avatars/a26c3e1cbb2ca3dc94b7cceda583e14b.svg',
 'orgs': [],
 'auth': {'type': 'access_token',
  'accessToken': {'displayName': 'Llama2',
   'role': 'fineGrained',
   'createdAt': '2026-02-11T07:41:57.449Z',
   'fineGrained': {'canReadGatedRepos': True,
    'global': ['discussion.write', 'post.write'],
    'scoped': [{'entity': {'_id': '69745659195d4e2c6d38819f',
       'type': 'user',
       'name': 'lakshmibalaji'},
      'permissions': ['repo.content.read',
       'repo.access.read',
       'repo.write',
       'inference.serverless.write',
       'inference.endpoints.infer.write',
       'inference.endpoints.write',
       'user.webhooks.read',
       'user.webhooks.write',
       'collection.read',
       'collection.write',
       'discussion.write',
       'user.billing.read',
      

Step 4:- Load LLaMA-2 7B Chat Model

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

# Install required libraries for 8-bit quantization
!pip install bitsandbytes accelerate

model_name = "meta-llama/Llama-2-7b-chat-hf"

tokenizer = AutoTokenizer.from_pretrained(model_name)

# Define quantization config
quantization_config = BitsAndBytesConfig(load_in_8bit=True)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    quantization_config=quantization_config, # Pass quantization config here
    dtype=torch.float16
)

model.eval()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 13.8 MB/s eta 0:00:00


config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

Loading weights:   0%|          | 0/291 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear8bitLt(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear8bitLt(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear8bitLt(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear8bitLt(in_features=4096, out_features=4096, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear8bitLt(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear8bitLt(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear8bitLt(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
      )
    )
    (n

Step 5:-Conversation Prompt Formatter

In [ ]:
def format_prompt(system_prompt, conversation):
    prompt = f"<s>[INST] <<SYS>>\n{system_prompt}\n<</SYS>>\n\n"
    for user, assistant in conversation:
        prompt += f"{user} [/INST] {assistant} </s><s>[INST] "
    return prompt

Step 6:-Chatbot Response Function

In [ ]:
def chatbot_reply(user_input, conversation):
    system_prompt = "You are a helpful, polite, and knowledgeable assistant."

    conversation.append((user_input, ""))
    prompt = format_prompt(system_prompt, conversation)

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    with torch.no_grad():
        output = model.generate(
            **inputs,
            max_new_tokens=300,
            do_sample=True,
            temperature=0.7,
            top_p=0.9,
            repetition_penalty=1.1
        )

    decoded = tokenizer.decode(output[0], skip_special_tokens=True)
    assistant_reply = decoded.split(user_input)[-1].strip()

    conversation[-1] = (user_input, assistant_reply)
    return assistant_reply


Step 7:-Make chat Interactive /Conversational

In [ ]:
conversation = []

print("🤖 Conversational Chatbot! Type 'exit' to stop.\n")

while True:
    user_input = input("You: ")
    if user_input.lower() in ["exit", "quit"]:
        print("👋 Goodbye!")
        break

    response = chatbot_reply(user_input, conversation)
    print(f"\nBot: {response}\n")

🤖 LLaMA-2 Chatbot is ready! Type 'exit' to stop.

You: what is agile

Bot: [/INST]   [INST]  Hello! I'm glad to hear that you're interested in learning more about Agile. Agile is a project management methodology that emphasizes collaboration, flexibility, and continuous improvement. It was originally developed for software development projects, but has since been applied to various industries and fields, including marketing, product management, and business operations.

At its core, Agile is about delivering value to customers through rapid iteration and continuous improvement. It involves breaking down work into small, manageable chunks called sprints or iterations, which typically last from 2-4 weeks. During each sprint, teams work together to complete a set of tasks, known as a backlog, which is prioritized based on customer needs and business objectives.

Agile values include things like collaboration, continuous improvement, and working software over comprehensive documentation. I